In [1]:
import pandas as pd
import numpy as np

In [2]:
linelist =pd.read_excel('Chikungunya linelistJuly_2025.xlsx')

In [3]:
linelist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 42 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   Column1                                                     613 non-null    float64
 1   In patienT/Outpatient                                       438 non-null    object 
 2   County                                                      613 non-null    object 
 3   Sub County                                                  609 non-null    object 
 4   Ward                                                        462 non-null    object 
 5   Village or Town and Neighbourhood INDICATE major landmarks  536 non-null    object 
 6   Health facility                                             597 non-null    object 
 7   Sex                                                         569 non-null    object 
 8   

Convert Date Seen to datetime

In [6]:
linelist['Date_seen'] = pd.to_datetime(linelist['Date seen (ddmmyyyy)'], format='%d%m%Y', errors='coerce')

linelist['Date_seen'].value_counts()

Date_seen
2025-06-06    32
2025-06-03    28
2025-06-05    27
2025-06-11    26
2025-06-07    25
              ..
2025-04-28     1
2025-05-23     1
2025-04-03     1
2025-04-13     1
2025-06-29     1
Name: count, Length: 84, dtype: int64

Clean age and create Age groups

Group 1: <60 , Group 2: >60

In [8]:
linelist = linelist.dropna(subset=['Date_seen', 'Age(years)'])

linelist['age_group']= np.where(linelist['Age(years)']<60, 'new_inf_1', 'new_inf_2 ')

Aggregate: Count cases per day per age group


In [10]:
observed_daily = linelist.groupby(['Date_seen','age_group']).size().unstack(fill_value=0).reset_index()

observed_daily

age_group,Date_seen,new_inf_1,new_inf_2
0,2025-03-14,1,0
1,2025-03-20,1,0
2,2025-03-22,0,1
3,2025-03-25,1,0
4,2025-03-26,0,1
...,...,...,...
72,2025-06-25,1,0
73,2025-06-26,2,0
74,2025-06-27,3,1
75,2025-06-28,2,0


Create the 'time' column (days since the first case)

In [12]:
start_date = observed_daily['Date_seen'].min()
observed_daily['time'] = (observed_daily['Date_seen'] - start_date).dt.days
observed_daily

age_group,Date_seen,new_inf_1,new_inf_2,time
0,2025-03-14,1,0,0
1,2025-03-20,1,0,6
2,2025-03-22,0,1,8
3,2025-03-25,1,0,11
4,2025-03-26,0,1,12
...,...,...,...,...
72,2025-06-25,1,0,103
73,2025-06-26,2,0,104
74,2025-06-27,3,1,105
75,2025-06-28,2,0,106


Final Selection and reordering for FME


Columns: time, new_inf_1, new_inf_2

In [14]:
fme_ready_data = observed_daily[['time', 'new_inf_1', 'new_inf_2 ']].sort_values('time')
fme_ready_data

age_group,time,new_inf_1,new_inf_2
0,0,1,0
1,6,1,0
2,8,0,1
3,11,1,0
4,12,0,1
...,...,...,...
72,103,1,0
73,104,2,0
74,105,3,1
75,106,2,0


Save to CSV. to load in R


In [15]:
fme_ready_data.to_csv('chikungunya_fme_ready_data.csv', index=False)

In [16]:
print("Data Transformation Complete. Sample:")
print(fme_ready_data.head())

Data Transformation Complete. Sample:
age_group  time  new_inf_1  new_inf_2 
0             0          1           0
1             6          1           0
2             8          0           1
3            11          1           0
4            12          0           1
